In [13]:
import requests
import os
from datetime import datetime, timedelta
import numpy as np
import json

In [14]:
with open('../configs/config.json', 'r') as file:
    current_project = json.load(file)["current_project"]

config_name = current_project['config_name']
start_date = current_project['start_date']
end_date = current_project['end_date']
weather_model = current_project['weather_model']

In [15]:
with open('config.json', 'r') as file:
    config = json.load(file)["simulation_config"][config_name]

In [16]:
with open('config.json', 'r') as file:
    weather_model_config = json.load(file)["weather_model_config"][weather_model]

#### Load data using API

In [17]:
def fetch_daily_data(base_url, start_date, end_date, lat0, long0, lat1, long1, folder_path):
    # Convert input dates to datetime objects
    start_dt = datetime.strptime(start_date, "%Y%m%d")
    end_dt = datetime.strptime(end_date, "%Y%m%d")

    variables = ['T_2M', 'U', 'V', 'GLOB', 'RELHUM_2M', 'PMSL', 'CLCT', 'PS']
    # Loop through each day within the range
    current_date = start_dt
    while current_date < end_dt:
        next_date = current_date + timedelta(days=1)

        if next_date > end_dt:
            next_date = end_dt

        # Format dates for the API request
        start_window_str = current_date.strftime("%Y%m%d")
        end_window_str = next_date.strftime("%Y%m%d")

        for var in variables:
            # Construct the API URL for each 1-day window
            url = f"{base_url}/{start_window_str}/{end_window_str}/{lat0}/{long0}/{lat1}/{long1}?variables={var}"
        
            # Fetch data from the API
            response = requests.get(url)
    
            if response.status_code == 200:
                data = response.json()
                
                # Save the data to a file with date range in the filename
                file_name = f"{start_window_str}_{end_window_str}_{var}.json"
                
                file_path = os.path.join(folder_path, file_name)
                
                with open(file_path, "w") as file:
                    file.write(response.text)
                    print(f"Saved data to {file_path}")
            else:
                print(f"Failed to fetch data for {start_window_str} to {end_window_str}. Error : {response.text}. url : {url}")

        current_date = next_date


In [18]:
# getting x,y min and max of the model grid
path_grid = config["grid_folder_path"]
with open(os.path.join(path_grid, 'lon_grid.npy'), 'rb') as f:
    long = np.load(f)
with open(os.path.join(path_grid, 'lat_grid.npy'), 'rb') as f:
    lat = np.load(f)

long0_icon_wgs84 = long.min() 
lat0_icon_wgs84 = lat.min()
long1_icon_wgs84 = long.max()
lat1_icon_wgs84 = lat.max()

In [19]:
buffer_icon_grid = 0.02 

base_url = weather_model_config['base_url']
folder_path = weather_model_config['raw_results_from_api_folder']

test = fetch_daily_data(base_url, start_date, end_date, lat0_icon_wgs84 - buffer_icon_grid, long0_icon_wgs84 - buffer_icon_grid, lat1_icon_wgs84 + buffer_icon_grid, long1_icon_wgs84 + buffer_icon_grid, folder_path)

Saved data to C:\Users\leroquan\Documents\00-Work_space\06-mit_gcm_geneva\data\COSMO reanalysis\raw_from_alplakes_api\20240301_20240302_T_2M.json
Saved data to C:\Users\leroquan\Documents\00-Work_space\06-mit_gcm_geneva\data\COSMO reanalysis\raw_from_alplakes_api\20240301_20240302_U.json
Saved data to C:\Users\leroquan\Documents\00-Work_space\06-mit_gcm_geneva\data\COSMO reanalysis\raw_from_alplakes_api\20240301_20240302_V.json
Saved data to C:\Users\leroquan\Documents\00-Work_space\06-mit_gcm_geneva\data\COSMO reanalysis\raw_from_alplakes_api\20240301_20240302_GLOB.json
Saved data to C:\Users\leroquan\Documents\00-Work_space\06-mit_gcm_geneva\data\COSMO reanalysis\raw_from_alplakes_api\20240301_20240302_RELHUM_2M.json
Saved data to C:\Users\leroquan\Documents\00-Work_space\06-mit_gcm_geneva\data\COSMO reanalysis\raw_from_alplakes_api\20240301_20240302_PMSL.json
Saved data to C:\Users\leroquan\Documents\00-Work_space\06-mit_gcm_geneva\data\COSMO reanalysis\raw_from_alplakes_api\2024030